# face 모델 시연
- 저장된 사진 혹은 카메라로 찍은 사진으로 모델을 돌려서 분위기 파악
- 파악된 분위기로 노래가 재생됨

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms

In [3]:
class my_model(nn.Module):
    def __init__(self):
        super(my_model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1)
                                       # 채널 수          커널수         커널 사이즈, 
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 8)   # class는 총 8개
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.view(-1, 8 * 8 * 16)    # 차원을 변경함
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x)
        return x

In [4]:
# 모델 불러오기
model = my_model()
model_file = './model/face_3.pth'
model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [10]:
# 시연할 새로운 이미지 준비
sample_image = './face/test/angry/PrivateTest_88305.jpg'
img = Image.open(sample_image)
img = np.array(img)
img = torch.FloatTensor(img)
img.shape

torch.Size([48, 48])

In [12]:
# 불러온 파일도 전처리 해준다.
preprocessing = transforms.Compose([
    transforms.Resize(size = (32, 32)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
img = img.unsqueeze(0)  # 차원을 추가
p_img = preprocessing(img)

print(p_img.shape)

RuntimeError: output with shape [1, 32, 32] doesn't match the broadcast shape [3, 32, 32]

In [93]:
# 모델 시연 라쓰고
model.eval()

with torch.no_grad():
    #p_img = p_img.unsqueeze(dim = 0)
    output = model(p_img)
    result = torch.argmax(output, dim = 1).item()
    #output = F.sigmoid(output).item()
print(f"output : {output}")
print(f"result : {result}")

output : tensor([[ -2.8829,  -4.9668,  -1.2449,  -1.3738,  -2.8516,  -1.5675,  -2.0431,
         -13.0762]])
result : 2


C:\Users\kdp\AppData\Local\Temp\ipykernel_8868\2343452117.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [94]:
# 분석된 결과에 따라 어떤 분위기/감정인지 알려주기
mood_dict = {'0':'angry', '1':'anxiety', '2':'depressed', '3':'dynamic', '4':'happy', '5':'peaceful', '6':'tired', '7':'withered'}
mood = mood_dict[f"{result}"].upper()
print(f"your image has {mood} mood")

your image has DEPRESSED mood


# 노래 추천

In [95]:
import sounddevice as sd
import soundfile as sf

# user_answer = int(input("당신의 mood와 어울리는 노래를 원한다면 1을, 환기하고 싶다면 2를 입력: "))
user_answer = 1

In [96]:
if user_answer == 1:
    mp3_path = './song_list/song_01.mp3'

    # MP3 파일 재생
    data, samplerate = sf.read(mp3_path)
    sd.play(data, samplerate)
    sd.wait()

elif user_answer == 2: pass
    
else : print('재입력하시오. 종료를 원한다면 q d입력')